In [1]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
import tensorflow.keras.losses

In [2]:
custom_objects = {"mse": tensorflow.keras.losses.MeanSquaredError()}

# Load the model with the custom loss object
model = load_model("image_reconstruction_model.h5", custom_objects=custom_objects)
model.summary()  # Verify the model's architecture

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ Input_X1            │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Input_X2            │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_72 (Conv2D)  │ (None, 128, 128,  │      1,792 │ Input_X1[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_74 (Conv2D)  │ (None, 128, 128,  │      1,792 │ Input_X2[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_26    │ (None, 64, 64,    │          0 │ conv2d_72[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_28    │ (None, 64, 64,    │          0 │ conv2d_74[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_73 (Conv2D)  │ (None, 64, 64,    │     73,856 │ max_pooling2d_26… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_75 (Conv2D)  │ (None, 64, 64,    │     73,856 │ max_pooling2d_28… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_27    │ (None, 32, 32,    │          0 │ conv2d_73[0][0]   │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_29    │ (None, 32, 32,    │          0 │ conv2d_75[0][0]   │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_4 (Flatten) │ (None, 131072)    │          0 │ max_pooling2d_27… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_5 (Flatten) │ (None, 131072)    │          0 │ max_pooling2d_29… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_14      │ (None, 262144)    │          0 │ flatten_4[0][0],  │
│ (Concatenate)       │                   │            │ flatten_5[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 256)       │ 67,109,120 │ concatenate_14[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 512)       │    131,584 │ dense_6[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 131072)    │ 67,239,936 │ dense_7[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_2 (Reshape) │ (None, 32, 32,    │          0 │ dense_8[0][0]     │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_76 (Conv2D)  │ (None, 32, 32,    │    147,584 │ reshape_2[0][0]   │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 134,855,045 (514.43 MB)

 Trainable params: 134,855,043 (514.43 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [3]:
image_size = (128, 128)

def load_image(image_path, size):
    """Load an image, convert to RGB, resize and normalize it."""
    return np.array(Image.open(image_path).convert('RGB').resize(size)) / 255.0

# Define folders for original images, ELA images, and ground truth masks
tp_folder = "C:/Users/vinay/Downloads/CASIA2.0_revised_corrected/casia/Tp"
ela_tp_folder = "C:/Users/vinay/Downloads/CASIA2.0_revised_corrected/casia/ELA_Tp"
gt_folder = "C:/Users/vinay/Downloads/CASIA2.0_revised_corrected/casia/Gt"
sample_limit = 1500

In [4]:
x1_data = []  # Original images
x2_data = []  # ELA images
y_data = []   # Masks

In [5]:
for file_name in os.listdir(tp_folder):
    if len(x1_data) >= sample_limit:
        break
    if file_name.lower().endswith(('jpg', 'bmp', 'tif', 'tiff', 'png')):
        # Build filenames based on the naming convention for ground truth
        base_name = file_name.rsplit('.', 1)[0]
        gt_name = base_name + "_gt.png"  # Ground truth mask file name

        # Build full paths for the images
        tp_path = os.path.join(tp_folder, file_name)
        ela_tp_path = os.path.join(ela_tp_folder, file_name)
        gt_path = os.path.join(gt_folder, gt_name)

        # Load original image and ELA image
        x1_data.append(load_image(tp_path, image_size))
        x2_data.append(load_image(ela_tp_path, image_size))

        # Load the corresponding mask; if not found, use an empty mask
        if os.path.exists(gt_path):
            mask = load_image(gt_path, image_size)
            y_data.append(mask)
            print("Loaded mask for:", file_name)
        else:
            y_data.append(np.zeros((image_size[0], image_size[1], 3)))  # Assuming RGB mask

# Convert lists to NumPy arrays
x1_data = np.array(x1_data)
x2_data = np.array(x2_data)
y_data = np.array(y_data)

Loaded mask for: Tp_D_CND_M_N_ani00018_sec00096_00138.tif
Loaded mask for: Tp_D_CND_M_N_art00076_art00077_10289.tif
Loaded mask for: Tp_D_CND_M_N_art00077_art00076_10290.tif
Loaded mask for: Tp_D_CND_S_N_ani00073_ani00068_00193.tif
Loaded mask for: Tp_D_CND_S_N_ind00078_ind00077_00476.tif
Loaded mask for: Tp_D_CND_S_N_txt00028_txt00006_10848.jpg
Loaded mask for: Tp_D_CNN_M_B_nat00056_nat00099_11105.jpg
Loaded mask for: Tp_D_CNN_M_B_nat10139_nat00059_11949.jpg
Loaded mask for: Tp_D_CNN_M_B_nat10139_nat00097_11948.jpg
Loaded mask for: Tp_D_CNN_M_N_ani00023_ani00024_10205.tif
Loaded mask for: Tp_D_CNN_M_N_ani00052_ani00054_11130.jpg
Loaded mask for: Tp_D_CNN_M_N_ani00057_ani00055_11149.jpg
Loaded mask for: Tp_D_CNN_M_N_arc00086_xxx00001_00306.tif
Loaded mask for: Tp_D_CNN_M_N_art00052_arc00030_11853.jpg
Loaded mask for: Tp_D_CNN_M_N_cha00026_cha00028_11784.jpg
Loaded mask for: Tp_D_CNN_M_N_nat00013_cha00042_11093.jpg
Loaded mask for: Tp_D_CNN_M_N_nat00041_nat10123_11439.jpg
Loaded mask fo

In [6]:
x1_train, x1_test, x2_train, x2_test, y_train, y_test = train_test_split(
    x1_data, x2_data, y_data, test_size=0.2, random_state=42
)

print(f"Training data size: {len(x1_train)}")
print(f"Testing data size: {len(x1_test)}")


Training data size: 1200
Testing data size: 300


In [7]:
def compute_iou(y_true, y_pred, threshold=0.5):
    """
    Compute the average Intersection over Union (IoU) for a set of images.

    Parameters:
      y_true (np.array): Ground truth masks (shape: [num_samples, height, width, channels])
      y_pred (np.array): Predicted masks (same shape as y_true)
      threshold (float): Threshold to binarize the masks

    Returns:
      float: Average IoU score across all samples.
    """
    # Binarize predictions and ground truth masks
    y_pred_bin = (y_pred > threshold).astype(np.uint8)
    y_true_bin = (y_true > threshold).astype(np.uint8)
    
    # If masks have 3 channels (RGB), use only one channel for IoU calculation
    if y_true_bin.ndim == 4 and y_true_bin.shape[-1] == 3:
        y_true_bin = y_true_bin[..., 0]
    if y_pred_bin.ndim == 4 and y_pred_bin.shape[-1] == 3:
        y_pred_bin = y_pred_bin[..., 0]
    
    ious = []
    for i in range(len(y_true_bin)):
        # Calculate the intersection and union for each sample
        intersection = np.logical_and(y_true_bin[i], y_pred_bin[i]).sum()
        union = np.logical_or(y_true_bin[i], y_pred_bin[i]).sum()
        iou = intersection / union if union != 0 else 1.0
        ious.append(iou)
        
    return np.mean(ious)

# ----------------------------------------------------------
# 4. Prediction & IoU Calculation
# ----------------------------------------------------------
# Get predictions from the model; adjust inputs if the model expects a list of two inputs
predictions = model.predict([x1_test, x2_test])

# Compute and print the average IoU score on the test set
iou_score = compute_iou(y_test, predictions, threshold=0.5)
print("Average IoU on test set:", iou_score)

10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 236ms/step
Average IoU on test set: 0.5134686485695509
